In [1]:
%load_ext autoreload
%autoreload 1
%aimport graph_description
from graph_description.datasets import get_knecht_data
import networkx as nx
import pysubgroup as ps
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [20]:
%aimport

Modules to reload:
graph_description

Modules to skip:



In [2]:
from graph_description.utils import get_dataset_folder

In [3]:
dataset_folder = get_dataset_folder()
data = np.load(dataset_folder/"citeseer.npz")

In [4]:
from scipy.sparse import coo_array

def npz_to_coo_array(data, prefix="attr"):
    """convert data from npz file into coo sparse array
    convert data from https://github.com/shchur/gnn-benchmark/tree/master/data
      into sparse matrix """
    assert prefix in ["adj", "attr"]
    def to_indices(data):
        n = len(data[prefix+"_indptr"]) -1
        return np.repeat(np.arange(n), np.diff(data[prefix+"_indptr"]))
    
    return coo_array((data[prefix+"_data"], (to_indices(data), data[prefix+"_indices"])), shape=data[prefix+"_shape"])

In [5]:
len(data["adj_data"])

4715

In [6]:
attr_array = npz_to_coo_array(data, "attr")
df = pd.DataFrame.sparse.from_spmatrix(attr_array)
df["label"] = data["labels"]

In [7]:
attr_array

<3312x3703 sparse array of type '<class 'numpy.float32'>'
	with 105165 stored elements in COOrdinate format>

In [8]:
adjacency = npz_to_coo_array(data, "adj")
G = nx.from_scipy_sparse_array(adjacency, create_using=nx.DiGraph)

In [9]:
adjacency

<3312x3312 sparse array of type '<class 'numpy.float32'>'
	with 4715 stored elements in COOrdinate format>

In [10]:
searchspace = ps.create_selectors(df, ignore=['label'])
searchspace = [sel for sel in searchspace if "==0" not in str(sel)]

In [11]:
from graph_description.networkx_aggregation import SumAggregator, MeanAggregator, apply_aggregator    

In [12]:
%load_ext snakeviz

In [13]:
dfx = df.iloc[:, :100]
dfx

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
isinstance(df[0].dtype, pd.SparseDtype)

True

In [15]:
type(searchspace[0])

pysubgroup.subgroup_description.EqualitySelector

In [16]:
df[0].array.sp_index.indices

array([  28,  338,  367,  438,  569,  586, 1403, 1445, 1551, 1562, 1830,
       1967, 2386, 2433, 2496, 2773, 2886, 3238])

In [17]:
searchspace[0].covers(dfx)

0       False
1       False
2       False
3       False
4       False
        ...  
3307    False
3308    False
3309    False
3310    False
3311    False
Name: 0, Length: 3312, dtype: Sparse[bool, False]

In [32]:
dir(dfx[0].sparse)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_delegate_accessors',
 '_delegate_method',
 '_delegate_property_get',
 '_delegate_property_set',
 '_parent',
 '_validate',
 '_validation_msg',
 'density',
 'fill_value',
 'from_coo',
 'npoints',
 'sp_values',
 'to_coo',
 'to_dense']

In [22]:
%autoreload

In [23]:
df1, searchspace1 = apply_aggregator(SumAggregator, dfx, G, searchspace[:100])
df2, searchspace2 = apply_aggregator((SumAggregator, MeanAggregator), df1, G)

init
prep done
init
prep done


C:\Users\felix\Desktop\projects\graph_description\src\graph_description\networkx_aggregation.py:67: RuntimeWarning: invalid value encountered in scalar divide
  arr[node] += column[other]


In [27]:
total_df = pd.concat([df, df1, df2], axis=1)
total_searchspace = searchspace+searchspace1+searchspace2

In [28]:
target = ps.BinaryTarget ('label', 1)

In [46]:
def prune_sparse_selectors(searchspace, df):
    out_sel = []
    for sel in searchspace:
        
        if not isinstance(sel, ps.EqualitySelector):
            out_sel.append(sel)
            continue
        if not isinstance(df[sel.attribute_name].dtype, pd.SparseDtype):
            out_sel.append(sel)
            continue
        if sel.attribute_value == df[sel.attribute_name].sparse.fill_value:
            continue
        out_sel.append(sel)
    return out_sel

In [47]:
ss2 = prune_sparse_selectors(total_searchspace, total_df)
print(len(total_searchspace), len(ss2))

4920 4620


In [48]:
total_searchspace[-10:]

[neigh_mean(neigh_sum(98==1.0))==0.0,
 neigh_mean(neigh_sum(98==1.0))==0.09090909090909091,
 neigh_mean(neigh_sum(99==1.0))==0.0,
 neigh_mean(neigh_sum(99==1.0))==1.0,
 neigh_mean(neigh_sum(all_ones)).isnull(),
 neigh_mean(neigh_sum(all_ones))<0.0,
 neigh_mean(neigh_sum(all_ones)): [0.0:0.1[,
 neigh_mean(neigh_sum(all_ones)): [0.1:0.5[,
 neigh_mean(neigh_sum(all_ones)): [0.5:1.0[,
 neigh_mean(neigh_sum(all_ones))>=1.0]

In [ ]:
task = ps.SubgroupDiscoveryTask (
    total_df,
    target,
    ss2,
    result_set_size=20,
    depth=3,
    qf=ps.GeneralizationAware_StandardQF(0.3),
    constraints=[ps.MinSupportConstraint(50)])
algorithm = ps.Apriori(ps.NumpySetRepresentation)
algorithm.use_vectorization=False
result = algorithm.execute(task)

C:\Users\felix\Desktop\projects\pysubgroup\src\pysubgroup\binary_target.py:546: RuntimeWarning: invalid value encountered in scalar divide
  tau_diff = pos / (pos + delta_n)


In [ ]:
result.to_dataframe()

In [ ]:
from pysubgroup.visualization import supportSetVisualization
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(supportSetVisualization(result))